In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
from pydub import AudioSegment

# Listening to existing samples

In [27]:
meta_sample = pd.read_csv('./samples/meta_samples.csv')

In [28]:
meta_sample.head(30)

,sample_name,crap_id,learner_name,format,genre,upscale_id
0,4520,1,1b,mp3,Electronic,1
1,4522,1,1b,mp3,Electronic,1
2,13710,1,1b,mp3,Electronic,1
3,14208,1,1b,mp3,Electronic,1
4,1075,1,1b,mp3,Experimental,1
5,1083,1,1b,mp3,Experimental,1
6,1277,1,1b,mp3,Experimental,1
7,1733,1,1b,mp3,Experimental,1
8,190,1,1b,mp3,Folk,1
9,200,1,1b,mp3,Folk,1


In [29]:
def get_sample(meta_sample, genre=None, crap_id=None, learner_name=None, upscale_id=None, n=None, return_sample=False):
    if genre is not None:
        meta_sample = meta_sample[meta_sample.genre == genre]
    if crap_id is not None:
        meta_sample = meta_sample[meta_sample.crap_id == crap_id]
    if learner_name is not None:
        meta_sample = meta_sample[meta_sample.learner_name == learner_name]
    if upscale_id is not None:
        meta_sample = meta_sample[meta_sample.upscale_id == upscale_id]
    if n is not None:
        sample_name, crap_id, learner_name, upscale_id, f = meta_sample.iloc[n][['sample_name', 'crap_id', 'learner_name', 'upscale_id', 'format']]
    else:
        sample_name, crap_id, learner_name, upscale_id, f = meta_sample.sample().iloc[0][['sample_name', 'crap_id', 'learner_name', 'upscale_id', 'format']]
    orig = AudioSegment.from_file(f'./samples/original/{sample_name}.{f}')
    crap = AudioSegment.from_file(f'./samples/crappified/{sample_name}_{crap_id}.{f}')
    reco = AudioSegment.from_file(f'./samples/reconstructed/{sample_name}_{crap_id}_{learner_name}_upscale{upscale_id}.{f}')
    print('Crappified file :')
    display(crap)
    print('Reconstructed file :')
    display(reco)
    print('Original file :')
    display(orig)
    if return_sample:
        return crap, reco, orig
    

In [31]:
get_sample(meta_sample, genre = 'Pop', n=0, upscale_id=2)

Crappified file :


Reconstructed file :


Original file :


# Creating new samples

In [4]:
from crystal_clear.upscale import upscale, upscale2, FeatureLoss
from fastai.basic_train import load_learner
from crystal_clear.prepare import path_mp3
from pathlib import Path
from fastai.vision import * 
from fastai import *

In [5]:
torch.cuda.is_available()

True

In [6]:
meta = pd.read_csv('./data/crappified/dataset_1/meta/meta_mp3.csv')
meta.head()

,track_id,n_window,rest,genre,subset
0,2,82,43,Hip-Hop,train
1,5,82,52,Hip-Hop,valid
2,10,82,43,Pop,train
3,140,82,43,Folk,train
4,141,82,43,Folk,train


In [7]:
learner_name = '1b.pkl'
learn = load_learner('./data/crappified/dataset_1/models_export/', learner_name)

In [8]:
genre_list = np.unique(meta.genre)
genre_list

array(['Electronic', 'Experimental', 'Folk', 'Hip-Hop', 'Instrumental', 'International', 'Pop', 'Rock'], dtype=object)

In [9]:
def get_track_id(genre=None, n=None):
    '''
    Get the id of a track in the validation set.
    If genre is specified, pick a track in the corresponding genre.
    If n is specified, get the n-th track corresponding to the arguments,
    else pick a random corresponding track'''
    meta_valid = meta[meta.subset == 'valid'].reset_index(drop=True)
    if genre is not None:
        meta_valid = meta_valid[meta_valid.genre == genre].reset_index(drop=True)
    if n is not None:
        return meta_valid.loc[n, 'track_id']
    else:
        return meta_valid.track_id.sample().iloc[0]

In [10]:
def inspect(track_id, upscale_id=1, display_music=True):
    path_orig = path_mp3(track_id)
    path_crap = Path(f'./data/crappified/dataset_1/mp3/{track_id}.mp3')
    orig = AudioSegment.from_mp3(path_orig)
    crap = AudioSegment.from_mp3(path_crap)
    if upscale_id == 1:
        reconstructed = upscale(path_crap, learn)
    elif upscale_id == 2:
        reconstructed = upscale2(path_crap, learn)
    else:
        raise Exception(f'upscale id {upscale_id} not recognized, at the moment only 1 and 2 supported')
    if not display_music:
        return orig, crap, reconstructed
    print('original')
    display(orig)
    print('crappified')
    display(crap)
    print('reconstructed')
    display(reconstructed)
    return orig, crap, reconstructed

In [20]:
track_id = get_track_id(n=2, genre='Rock')
orig, crap, reconstructed = inspect(track_id, upscale_id=2)

gpu is used for prediction


/home/ludovic/Documents/projects_ML/Crystal_Clear/crystal_clear/upscale.py:148: RuntimeWarning: invalid value encountered in true_divide
  data = data / m
100%|██████████| 330/330 [00:45<00:00,  4.61it/s]
/home/ludovic/Documents/projects_ML/Crystal_Clear/crystal_clear/upscale.py:170: RuntimeWarning: invalid value encountered in true_divide
  angle = (Zxx / np.abs(Zxx))


original


crappified


reconstructed


In [24]:
def add_sample(sample_name, orig, crap, reconstructed, learner_name, crap_id=1, upscale_id=1, format='mp3', genre=''):
    try:
        df_meta_samples = pd.read_csv('./samples/meta_samples.csv')
        new_meta = df_meta_samples.append({'sample_name':sample_name, 'crap_id':crap_id,
                                           'learner_name': learner_name, 'format':format,
                                           'genre':genre, 'upscale_id':upscale_id}, ignore_index=True)
    except:
        new_meta = pd.DataFrame({'sample_name':sample_name, 'crap_id':crap_id,
                                 'learner_name':learner_name, 'format':format,
                                 'genre':genre, 'upscale_id':upscale_id}, index= [0])
    new_meta.to_csv('./samples/meta_samples.csv', index=None)
    orig.export(f'./samples/original/{sample_name}.{format}', format=format)
    crap.export(f'./samples/crappified/{sample_name}_{crap_id}.{format}', format=format)
    reconstructed.export(f'./samples/reconstructed/{sample_name}_{crap_id}_{learner_name}_upscale{upscale_id}.{format}', format=format)

In [26]:
for i in range(1,3):
    upscale_id = i
    for genre in genre_list:
        for i in range(4):
            track_id = get_track_id(n=i, genre=genre)
            orig, crap, reconstructed = inspect(track_id, upscale_id=upscale_id, display_music=False)
            add_sample(f'{track_id}', orig, crap, reconstructed, learner_name=os.path.splitext(learner_name)[0], upscale_id=upscale_id, genre=genre)

gpu is used for prediction


100%|██████████| 3/3 [00:00<00:00,  6.36it/s]


gpu is used for prediction


100%|██████████| 3/3 [00:00<00:00,  6.50it/s]


gpu is used for prediction


...

/home/ludovic/Documents/projects_ML/Crystal_Clear/crystal_clear/upscale.py:148: RuntimeWarning: invalid value encountered in true_divide
  data = data / m
100%|██████████| 330/330 [00:52<00:00,  6.32it/s]
/home/ludovic/Documents/projects_ML/Crystal_Clear/crystal_clear/upscale.py:170: RuntimeWarning: invalid value encountered in true_divide
  angle = (Zxx / np.abs(Zxx))


gpu is used for prediction


/home/ludovic/Documents/projects_ML/Crystal_Clear/crystal_clear/upscale.py:148: RuntimeWarning: invalid value encountered in true_divide
  data = data / m
100%|██████████| 330/330 [00:54<00:00,  5.12it/s]
/home/ludovic/Documents/projects_ML/Crystal_Clear/crystal_clear/upscale.py:170: RuntimeWarning: invalid value encountered in true_divide
  angle = (Zxx / np.abs(Zxx))


gpu is used for prediction


100%|██████████| 330/330 [00:54<00:00,  6.05it/s]
